In [ ]:


손실함수를 최소화하는 경사하강법이다.
각 단계에서 손실함수의 기울기(잔차)를 예측하는 결정트리를 하나씩 추가

[1] 기본 학습<hr>

In [ ]:
## 모듈 로딩
import xgboost as xgb
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

## 1) 데이터 로드
X, y = load_breast_cancer(return_X_y=True)

## 2) 학습/테스트 분리
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

## 3) sklearn 래퍼 모델 생성 (XGBClassifier)
model = xgb.XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.9,
    colsample_bytree=0.9,
    tree_method="hist",     # 기본적으로 빠르고 실무에서 많이 씀
    # GPU 사용 시 아래 한 줄만 켜세요 (CUDA 환경일 때)
    # device="cuda",
    random_state=42,
    eval_metric="logloss",
)

## 4) 학습 (sklearn처럼 fit)
model.fit(X_train, y_train)

## 5) 예측/평가 (sklearn처럼 predict, score)
pred = model.predict(X_test)
acc = accuracy_score(y_test, pred)

print("Accuracy:", acc)
print(classification_report(y_test, pred))


Accuracy: 0.956140350877193
              precision    recall  f1-score   support

           0       0.97      0.90      0.94        42
           1       0.95      0.99      0.97        72

    accuracy                           0.96       114
   macro avg       0.96      0.95      0.95       114
weighted avg       0.96      0.96      0.96       114



[2] 조기 종료 기능 추가 <hr>

In [2]:
## 모듈 로딩
import xgboost as xgb
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

## 1) 데이터
X, y = load_breast_cancer(return_X_y=True)

## 2) Train/Test 분리
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

## 3) 조기종료용 Validation 분리 (Train 내부에서 다시 쪼갬)
X_tr, X_val, y_tr, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

## 4) EarlyStopping 콜백 (patience=20 라운드)
early_stop = xgb.callback.EarlyStopping(
    rounds=20,
    save_best=True,          # 베스트 모델을 저장(실전에서 추천)
    metric_name="logloss",   # eval_metric과 맞춰주기
    data_name="validation_0" # eval_set의 첫 번째 데이터 이름
)

## 5) 모델 (sklearn 래퍼)
model = xgb.XGBClassifier(
    n_estimators=2000,       # 넉넉히 크게 주고 early stopping으로 자동 결정
    learning_rate=0.05,
    max_depth=4,
    subsample=0.9,
    colsample_bytree=0.9,
    tree_method="hist",
    # GPU 사용 시 아래 한 줄만 켜세요 (CUDA 환경일 때)
    # device="cuda",
    random_state=42,
    eval_metric="logloss",
    callbacks=[early_stop],
)

## 6) 학습: eval_set에 validation을 넣어야 조기종료가 동작
model.fit(
    X_tr, y_tr,
    eval_set=[(X_val, y_val)],
    verbose=False
)

print("best_iteration:", model.best_iteration)

## 7) 평가
pred = model.predict(X_test)   # sklearn 래퍼는 early stopping 시 best_iteration 기반 예측 동작을 안내함 :contentReference[oaicite:1]{index=1}
acc = accuracy_score(y_test, pred)

print("Accuracy:", acc)
print(classification_report(y_test, pred))


best_iteration: 77
Accuracy: 0.9385964912280702
              precision    recall  f1-score   support

           0       0.91      0.93      0.92        42
           1       0.96      0.94      0.95        72

    accuracy                           0.94       114
   macro avg       0.93      0.94      0.93       114
weighted avg       0.94      0.94      0.94       114

